In [1]:
from itertools import combinations
import numpy as np
import os
import pandas as pd
import random

In [2]:
POS = (("Q", 1), ("R", 2), ("W", 2), ("T", 1), (("R", "W", "T"), 2))
BENCH = 7
MONEY_COLS = ["$VORP", "$MG", "$VOLS"]
TOTAL_MONEY = 200

### Find cheat sheet in most recent directory

In [3]:
# Get folder for most recent season
folders = [
    f
    for f in os.listdir()
    if os.path.isdir(f) and not f.startswith(".") and not f.startswith("_")
]
newest_folder = sorted(folders)[-1]
assert "cheat_sheet.csv" in os.listdir(newest_folder), "No cheat sheet found"

### Preprocess/clean up cheat sheet

In [4]:
# Load cheat sheet
cheat_sheet = pd.read_csv(f"{newest_folder}/cheat_sheet.csv", header=5)

# Remove unnamed columns
cheat_sheet = cheat_sheet.loc[:, ~cheat_sheet.columns.str.contains("^Unnamed")]

# # Split all column names on periods and take the first part
cheat_sheet.columns = cheat_sheet.columns.str.split(".").str[0]

# Melt on duplicate columns
cols = cheat_sheet.columns[~cheat_sheet.columns.duplicated()]
n_splits = cheat_sheet.shape[1] // len(cols)
splits = np.split(np.arange(cheat_sheet.shape[1]), n_splits)
dfs = [cheat_sheet.iloc[:, split] for split in splits]
cheat_sheet = pd.concat(dfs, axis=0).reset_index(drop=True)

# Only take first character for values in POS column
cheat_sheet["POS"] = cheat_sheet["POS"].str[0]

# Remove unneeded positions
pos_needed = set(
    np.hstack(np.array([p[0] for p in POS], dtype=object)).tolist()
)
cheat_sheet = cheat_sheet[cheat_sheet["POS"].isin(pos_needed)]

# Remove dollar signs and convert to integers for $ columns
for col in MONEY_COLS:
    cheat_sheet[col] = cheat_sheet[col].str.replace("$", "").astype(int)

### Determine good team splits

In [5]:
def generate_sums(bins, counts):
    """
    Recursively generates all possible combinations of elements from bins
    based on the provided counts and sums them.

    Args:
        bins (list of lists): Each inner list represents a bin containing
                              elements to choose from.
        counts (list of ints): Each integer represents the number of elements
                               to select from the corresponding bin.

    Returns:
        dict: A dictionary where the keys are the sums of the combinations
              and the values are the combinations themselves.
    """
    # Base case: If there's only one bin, generate all combinations from it.
    if len(bins) == 1:
        return {sum(comb): comb for comb in combinations(bins[0], counts[0])}

    # Recursive case: Split bins into two halves.
    mid = len(bins) // 2
    left_bins = bins[:mid]
    right_bins = bins[mid:]
    left_counts = counts[:mid]
    right_counts = counts[mid:]

    # Generate sums (combinations) for the left and right halves.
    left_sums = generate_sums(left_bins, left_counts)
    right_sums = generate_sums(right_bins, right_counts)

    # Combine the left and right sums by adding their totals.
    combined_sums = {}
    for l_sum, l_comb in left_sums.items():
        for r_sum, r_comb in right_sums.items():
            total_sum = l_sum + r_sum
            combined_sums[total_sum] = l_comb + r_comb

    return combined_sums


def find_combinations(bins, counts, target_sum, shuffle=False):
    """
    Finds the combination of elements from bins that has the sum closest to
    the target sum.

    Args:
        bins (list of lists): Each inner list represents a bin containing
                              elements to choose from.
        counts (list of ints): Each integer represents the number of elements
                               to select from the corresponding bin.
        target_sum (int or float): The target sum to get as close to as
                                   possible with the combination.
        shuffle (bool): If True, shuffle the elements within each bin before
                        generating combinations. This introduces randomness
                        and can lead to different results on different runs.

    Returns:
        tuple: The combination of elements that has the sum closest to the
               target sum.
    """
    # Shuffle the elements within each bin if shuffle is True.
    if shuffle:
        for bin in bins:
            random.shuffle(bin)

    # Generate all possible sums and their corresponding combinations.
    possible_sums = generate_sums(bins, counts)

    # Initialize variables to track the closest combination.
    closest_diff = float("inf")
    closest_combination = None

    # Iterate through each combination to find the one closest to the target.
    for total_sum, combination in possible_sums.items():
        diff = abs(total_sum - target_sum)  # Calculate the difference.
        # Update the closest combination if this one is closer to the target.
        if diff < closest_diff:
            closest_diff = diff
            closest_combination = combination

    return closest_combination

In [6]:
# Set up variables
value = "$VORP"
target_sum = TOTAL_MONEY - BENCH
num_starters = np.sum([p[1] for p in POS])
nonzero = cheat_sheet[cheat_sheet[value] > 0]

# Split into bins
bins, counts = [], []
for pos, count in POS:
    if isinstance(pos, str):
        pos = [pos]
    bins.append(nonzero[nonzero["POS"].isin(pos)][value].values)
    counts.append(count)

# Find closest combination
closest_combination = find_combinations(bins, counts, target_sum, shuffle=True)

# Get players
player_idx = 0
team = []
for pos, count in POS:
    if isinstance(pos, str):
        pos = [pos]
    for _ in range(count):
        # Get player value
        val = closest_combination[player_idx]

        # Get players for this position
        pos_players = nonzero[nonzero["POS"].isin(pos)]

        # Find player in position with value
        player = pos_players[pos_players[value] == val].iloc[0]

        # Add player to team
        team.append(player)

        # Remove player from pool
        nonzero = nonzero.drop(player.name)

        # Increment player index
        player_idx += 1
pd.DataFrame(team)

,NAME,TM/BW,POS,$VORP,$MG,$VOLS,$RK,ECR,ADP
166,Jalen Hurts,PHI/5,Q,21,34,47,4.01,1.02,4.08
1,Breece Hall,NYJ/12,R,37,43,64,1.02,2.07,1.04
0,Christian McCaffrey,SF/9,R,48,60,92,1.01,1.07,1.01
84,Tyreek Hill,MIA/6,W,34,50,63,1.05,2.02,1.02
87,Justin Jefferson,MIN/6,W,27,38,43,2.02,2.08,1.07
200,Sam LaPorta,DET/5,T,14,16,21,6.01,6.04,4.03
34,Ezekiel Elliott,DAL/7,R,6,0,0,12.03,17.06,14.03
119,Jayden Reed,GB/10,W,6,3,0,12.04,12.06,11.01
